In [1]:
import json
from PIL import Image
import torch
from torchvision import transforms
from torchvision import models
import numpy as np
import glob
import time

|    *Name*         |*# Params*|*Top-1 Acc.*|*GPU time?*|
|:-----------------:|:--------:|:----------:|:-----------:|
| `efficientnet-b0` |   5.3M   |    76.3    |             |
| `efficientnet-b1` |   7.8M   |    78.8    |             |
| `efficientnet-b2` |   9.2M   |    79.8    |             |
| `efficientnet-b3` |    12M   |    81.1    |             |
| `efficientnet-b4` |    19M   |    82.6    |             |
| `efficientnet-b5` |    30M   |    83.3    |             |
| `efficientnet-b6` |    43M   |    84.0    |             |
| `efficientnet-b7` |    66M   |    84.4    |             |
| `Res18`           |    ?     |    69.4    |             |
| `Res34`           |    ?     |    73.3    |             |
| `Res50`           |    ?     |    76.1    |             |
| `Res101`          |    ?     |    76.3    |             |

In [2]:
def eval(model, device="cuda"):
    evaltime = []
    imgpath="../VOCdevkit/VOC2007/JPEGImages/"
    images=glob.glob(imgpath+"*jpg")
    for img in images[0:100]:
        # Preprocess image
        tfms = transforms.Compose([transforms.Resize(224), transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),])
        img = tfms(Image.open(img)).unsqueeze(0).to(device)
        #print(img.shape) # torch.Size([1, 3, 224, 224])

        # Load ImageNet class names
        labels_map = json.load(open('labels_map.txt'))
        labels_map = [labels_map[str(i)] for i in range(1000)]

        # Classify
        model.eval()
        tick = time.time()
        with torch.no_grad():
            outputs = model(img)
        #print("eval time=", time.time()-tick)
        evaltime.append(time.time()-tick)

        # Print predictions
        #print('-----')
        #for idx in torch.topk(outputs, k=5).indices.squeeze(0).tolist():
        #    prob = torch.softmax(outputs, dim=1)[0, idx].item()
        #    print('{label:<75} ({p:.2f}%)'.format(label=labels_map[idx], p=prob*100))
    print("mean eval time:", np.mean(evaltime))

# setup efficientnet

In [ ]:
!pip install --upgrade efficientnet-pytorch

Requirement already up-to-date: efficientnet-pytorch in /home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages (0.5.1)


## Inference with GPUs

In [ ]:
from efficientnet_pytorch import EfficientNet
model = EfficientNet.from_pretrained('efficientnet-b0').to("cuda")

In [ ]:
eval(model)

## Inference with CPUs

In [ ]:
model = model.to("cpu")

In [ ]:
eval(model, "cpu")

# setup and eval resnet

In [ ]:
model = models.resnet50(pretrained=True).to("cuda")

## Inference with GPUs

In [ ]:
eval(model)

## Inference with CPUs

In [ ]:
model = model.to("cpu")

In [ ]:
eval(model, "cpu")